In [1]:
import tensorflow as tf
import numpy as np

X_train = np.arange(10).reshape((10,1))
y_train = np.array([1.0,1.3,3.1,2.0,5.0,6.3,6.6,7.4,8.0,9.0])

class TfLinreg(object):
    def __init__(self,learning_rate = 0.01):
        self.w = tf.Variable(tf.zeros(shape = (1)))
        self.b = tf.Variable(tf.zeros(shape=(1)))
        self.optimizer = tf.keras.optimizers.SGD(lr = learning_rate)
    def fit(self,X,y,num_epochs = 10):
        training_costs = []
        for step in range(num_epochs):
            with tf.GradientTape as tape:
                z_net = self.w * X + self.b
                Z_net = tf.reshape(z_net,[-1])
                sqr_errors = tf.square(y - z_net)
                mean_cost = tf.reduce_mean(sqr_errors)
            grads = tape.gradient(mean_cost,[self.w,self.b])
            self.optimizer.apply_gradients(zip(grads,[self.w,self.b]))
            training_costs.append(mean_cost.numpy())
        return training_costs

(X_train,y_train),(X_test,y_test) = tf.keras.datasets.mnist.load_data()



11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)


In [3]:
X_train = X_train.reshape((60000,-1))
X_test = X_test.reshape((10000,-1))
mean_vals = np.mean(X_train,axis = 0)
std_val = np.std(X_train)
X_train_centered = (X_train - mean_vals) / std_val
X_test_centered = (X_test - mean_vals) / std_val
del X_train,X_test
np.random.seed(123)

In [4]:
y_train_onehot = tf.keras.utils.to_categorical(y_train)


In [5]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=50,input_dim = X_train_centered.shape[1],kernel_initializer="glorot_uniform",bias_initializer='zeros',activation='tanh'))
model.add(tf.keras.layers.Dense(units=50,input_dim = 50,kernel_initializer="glorot_uniform",bias_initializer='zeros',activation='tanh'))
model.add(tf.keras.layers.Dense(units=y_train_onehot.shape[1],input_dim = X_train_centered.shape[1],kernel_initializer="glorot_uniform",bias_initializer='zeros',activation='softmax'))
model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                39250     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                510       
Total params: 42,310
Trainable params: 42,310
Non-trainable params: 0
_________________________________________________________________


In [8]:
sgd_optimizer = tf.keras.optimizers.SGD(lr = 0.001,decay = 1e-7,momentum=0.9)
model.compile(optimizer = sgd_optimizer,loss = "categorical_crossentropy")
history = model.fit(X_train_centered,y_train_onehot,batch_size=64,epochs = 50,verbose = 1,validation_split=0.1)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/50
844/844 [==============================] - 2s 2ms/step - loss: 0.7234 - val_loss: 0.3585
Epoch 2/50
844/844 [==============================] - 2s 2ms/step - loss: 0.3651 - val_loss: 0.2710
Epoch 3/50
844/844 [==============================] - 2s 2ms/step - loss: 0.2989 - val_loss: 0.2344
Epoch 4/50
844/844 [==============================] - 2s 2ms/step - loss: 0.2622 - val_loss: 0.2102
Epoch 5/50
844/844 [==============================] - 2s 2ms/step - loss: 0.2366 - val_loss: 0.1941
Epoch 6/50
844/844 [==============================] - 2s 2ms/step - loss: 0.2172 - val_loss: 0.1815
Epoch 7/50
844/844 [==============================] - 2s 2ms/step - loss: 0.2015 - val_loss: 0.1728
Epoch 8/50
844/844 [==============================] - 2s 2ms/step - loss: 0.1884 - val_loss: 0.1638
Epoch 9/50
844/844 [==============================] - 2s 2ms/step - loss: 0.1772 - val_loss: 0.1577
Epoch 10/50
844/844 [==============================] - 2s 2ms/step - loss: 0.1674 - val_loss: 0.1516

In [29]:
y_train_pred = model.predict(X_train_centered,verbose = 0)
y_train_pred_check = []
for i in range(y_train_pred.shape[0]):
  y_train_pred_check.append(np.argmax(y_train_pred[i]))
correct_preds = np.sum(y_train == y_train_pred_check,axis = 0)
train_acc = correct_preds/y_train.shape[0]
print(train_acc*100)

98.98666666666666
